<a href="https://colab.research.google.com/github/Pulsar-kkaturi/DL-Education/blob/master/VisionDL_Lecture/Lecture2_MathofNN_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 2. 신경망의 이해

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

## 1. 신경망을 위한 데이터 표현

### 1.1. 텐서

텐서 표현

In [ ]:
# 스칼라
x1 = np.array(12)
print(f'scalar x1 = {x1}  (rank-{x1.ndim} tensor)')
# 벡터
x2 = np.array([3, 7, 9, 8, 1])
print(f'vector x2 = {x2}  (rank-{x2.ndim} tensor)')
# 행렬
x3 = np.array([[1,1,1], [2,2,2], [1,2,3]])
print(f'matrix x3 = {x3}  (rank-{x3.ndim} tensor)')

텐서의 핵심 속성

In [ ]:
# MNIST 데이터
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
# MNIST 데이터의 핵심 속성
print('축의 개수(랭크) = ', train_images.ndim)
print('크기 = ', train_images.shape)
print('데이터 타입 = ', train_images.dtype)

In [ ]:
# MNIST 데이터의 배치(batch) 분류
batch_list = []
for idx in range(6):
  batch_list.append(train_images[10000*idx:10000*(idx+1)])
print('총 배치의 수 = ', len(batch_list))
print('각 배치의 크기 = ', batch_list[0].shape)

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.title(f'batch_{i+1} sample')
  plt.imshow(batch_list[i][0])

### 1.2. 텐서 연산

**Dense 층 직접 구현**

output = relu(dot(W, input) + b)

In [ ]:
# relu 함수
def naive_relu(x):
    assert len(x.shape) == 2
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [ ]:
# sum 함수
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

직접 구현한 Dense 층 연산

In [ ]:
# 실험용 데이터 생성
x = np.random.random((20, 100))
y = np.random.random((20, 100))
print(len(x), x[0])

In [ ]:
# Dense 정방향 계산
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("걸린 시간: {0:.2f} s".format(time.time() - t0))

In [ ]:
# numpy 이용 시
t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.)
print("걸린 시간: {0:.2f} s".format(time.time() - t0))

## 2. 선형 분류 (linear classification) 실습

### 2.1. 데이터 생성

2D 평면에 두 클래스의 랜덤한 포인트 생성하기

In [ ]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(
    mean=[0, 3], # 포인트 집단의 평균
    cov=[[1, 0.5],[0.5, 1]], # 포인트 집단의 공분산
    size=num_samples_per_class) # 첫번째 클래스의 포인트 생성
positive_samples = np.random.multivariate_normal(
    mean=[3, 0],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class) # 다른 평균과 공분산을 가진 두번째 클래스

In [ ]:
print(positive_samples.shape)
print(positive_samples[0])

두 클래스를 (2000, 2) 크기의 하나의 배열로 쌓기

In [ ]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
print(inputs.shape)

(0과 1로 구성된) 정답지 생성하기

In [ ]:
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"),
                     np.ones((num_samples_per_class, 1), dtype="float32")))

In [ ]:
print(targets.shape)
print(targets[0], targets[1000])

두 클래스의 포인트를 그래프로 그리기

In [ ]:
plt.scatter(inputs[:, 0], inputs[:, 1], c=targets[:, 0])
plt.show()

### 2.2. 모델 설계

선형 분류기의 가중치 변수 만들기 (W, b)

In [ ]:
input_dim = 2 # 입력은 2D 포인트
output_dim = 1 # 출력 예측은 샘플당 하나의 점수 (0~1)
W = tf.Variable(initial_value=tf.random.uniform(shape=(input_dim, output_dim))) # kernel parameter
b = tf.Variable(initial_value=tf.zeros(shape=(output_dim,))) # bias parameter

정방향 패스 함수 (선형 분류이므로 연산은 단순한 선형 변환식이다)

y = W * x + b

In [ ]:
def model(inputs):
  return tf.matmul(inputs, W) + b
# tf.matmul() 함수는 점곱을 수행하는 텐서플로 내장함수

손실함수 = 평균 제곱 오차 함수 (Mean Squared Error; MSE)

In [ ]:
def square_loss(targets, predictions):
    per_sample_losses = tf.square(targets - predictions) # 각 샘플별 제곱(tf.square()) 오차
    return tf.reduce_mean(per_sample_losses) # 전체 샘플별 제곱 오차의 평균(tr.reduce_mean()) = MSE

훈련 스텝 함수

In [ ]:
learning_rate = 0.1 # 학습률

def training_step(inputs, targets):
    with tf.GradientTape() as tape: # 계산 그래프 생성
        predictions = model(inputs) # 계산 그래프 안에 모델의 정방향 패스 배치
        loss = square_loss(targets, predictions) # 손실 계산
    grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b]) # 가중치에 대한 손실의 Gradient 계산
    W.assign_sub(grad_loss_wrt_W * learning_rate) # 가중치(W) 업데이트
    b.assign_sub(grad_loss_wrt_b * learning_rate) # 가중치(b) 업데이트
    return loss

### 2.3. 모델 훈련

In [ ]:
for step in range(40): # 40번 학습
    loss = training_step(inputs, targets)
    print(f"{step+1}번째 스텝의 손실: {loss:.4f}") # 각 학습 epoch 별 손실 출력

### 2.4. 결과 분석

모델 예측

In [ ]:
predictions = model(inputs)
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5, cmap='viridis')
plt.show()

정답지와 비교

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.title('prediction')
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5, cmap='viridis')
plt.subplot(1,2,2)
plt.title('target')
plt.scatter(inputs[:, 0], inputs[:, 1], c=targets[:, 0])

선형분류기의 경계면 그리기

In [ ]:
x = np.linspace(-1, 4, 100) # 경계선 x축 리스트
y = - W[0] /  W[1] * x + (0.5 - b) / W[1] # 경계선 함수
plt.plot(x, y, "-r")
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5)
plt.show()

## 3. 밑바닥부터 딥러닝 구현하기 - Lecture 1 예제 재구현

### 3.1. 데이터 만들기 + 전처리: MNIST

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [ ]:
# 텐서플로로 구현한 모델
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

### 3.2. 모델 설계

In [ ]:
# Dense 층 구현
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value) # 무작위 초기화된 W 행렬

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value) # 무작위 초기화된 b 벡터

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b) # 정방향 패스 실행

    @property
    def weights(self):
        return [self.W, self.b] # 가중치 (W, b) 추출용 함수

In [ ]:
# 단순한 직렬 신경망 구현 (Sequential 모델)
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
           x = layer(x)
        return x # 들어온 레이어들을 쌓아서 신경망 구성

    @property
    def weights(self):
       weights = []
       for layer in self.layers:
           weights += layer.weights
       return weights # 신경망 전체의 가중치 추출

In [ ]:
# 직접 신경망 구현하기
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [ ]:
# 배치 제네레이터 (전체 MNIST 데이터셋에서 배치 추출)
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128): # 128 단위로 배치 추출
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

### 3.3. 훈련 시작

In [ ]:
learning_rate = 1e-3 # 학습률

def update_weights(gradients, weights): # 가중치 업데이트
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate) # 현 가중치에 - gradient * learning_rate

In [ ]:
# 하나의 훈련 스텝 정의
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape: # 계산 그래프 선언
        predictions = model(images_batch) # 정방향 패스 실행
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions) # 손실 함수 = categorical crossentropy
        average_loss = tf.reduce_mean(per_sample_losses) # 전체 모델의 손실
    gradients = tape.gradient(average_loss, model.weights) # 가중치에 대한 손실의 gradient 계산
    update_weights(gradients, model.weights) # gradient를 사용하여 가중치 업데이트
    return average_loss

In [ ]:
# 전체 훈련 루프 선언
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs): # 총 epoch에 따라 위에서 정의한 훈련 스텝 반복
        print(f"에포크 {epoch_counter}")
        batch_generator = BatchGenerator(images, labels) # 앞서 정의 배치 제네레이터에서 배치 추출
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch) # 훈련 스텝을 통해 손실 계산 및 가중치 업데이트
            if batch_counter % 100 == 0:
                print(f"{batch_counter}번째 배치 손실: {loss:.2f}")

In [ ]:
# 훈련 시작!
fit(model, train_images, train_labels, epochs=10, batch_size=128)

### 3.4. 모델 평가하기

In [ ]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1) # 최대값을 가진 index 추출 = 예측한 레이블
matches = predicted_labels == test_labels # 예측값 = 정답일 경우 1로 저장, 불칠치할 경우 0으로 저장
print(f"정확도: {matches.mean():.2f}")

In [ ]:
# 샘플 하나에 대한 예측 (0번)
print(predictions[0])
print(predicted_labels[0])
print(test_labels[0])
print(matches[0])

수고하셨습니다!